In [1]:
# Config

BENCHMARK = "TPCH"

if BENCHMARK == "TPCH":
    STATISTICS_PATH = 'TPC-H__SF_1.000000__RUNS_10'
    CHUNK_SIZE = 25000
elif BENCHMARK == "TPCDS":
    STATISTICS_PATH = 'TPC-DS__SF_1.000000__RUNS_1'
    CHUNK_SIZE = 100000
else:
    raise Exception("Unknown benchmark: " + BENCHMARK)
    
import pandas as pd
import numpy as np
from functools import reduce
import itertools

print(f"Model is configured for {BENCHMARK}")

Model is configured for TPCH


In [2]:
# Load table scan statistics

path = f"{STATISTICS_PATH}/table_scans.csv"
scans = pd.read_csv(path, sep='|')
EXPECTED_SCAN_COUNT = len(scans)
LOADED_BENCHMARK = BENCHMARK
print(f"Successfully loaded {path}")

def assert_correct_statistics_loaded():
    assert BENCHMARK == LOADED_BENCHMARK, f"The model is configured to use {BENCHMARK}, but {LOADED_BENCHMARK} is currently loaded.\nEither change the benchmark or re-run all cells"
    assert EXPECTED_SCAN_COUNT == len(scans), f"There should be {EXPECTED_SCAN_COUNT} table scans, but there are only {len(scans)}\nProbably one of the last commands reassigned it unintentionally"

Successfully loaded TPC-H__SF_1.000000__RUNS_10/table_scans.csv


In [3]:
# Validate table scans
assert_correct_statistics_loaded()

# To make sure pruning was not active,
# first fetch table sizes,
table_statistics = pd.read_csv(f"{STATISTICS_PATH}/table_meta_data.csv", sep='|')
table_sizes = dict(zip(table_statistics.table_name, table_statistics.row_count))

# then make sure INPUT_ROWS == table_size
def input_size_matches(row):
    #print(row)
    
    actual_row_count = row['INPUT_ROWS']
    table = row['TABLE_NAME']
    expected_row_count = table_sizes[table]
    return expected_row_count == actual_row_count

data_scans = scans[scans['COLUMN_TYPE'] == 'DATA']
input_size_matches = data_scans.apply(input_size_matches, axis=1)
all_sizes_match = reduce(np.logical_and, input_size_matches) #input_size_matches.apply()

if not all_sizes_match:
    raise Exception("The given statistics were probably created while pruning was active")
else:
    print("OK - looks like pruning was deactivated while the statistics were created")

OK - looks like pruning was deactivated while the statistics were created


In [4]:
# Cleanse table scans
assert_correct_statistics_loaded()

print(f"Statistics for {BENCHMARK} contain {len(scans)} table scans")

# If there is only one chunk, we cannot really prune
# TODO should only affect data segments?
scans = scans[scans['INPUT_ROWS'] > CHUNK_SIZE]
print(f"Of those, only {len(scans)} operate on more than {CHUNK_SIZE} tuples")

# Like scans are not useful if they start with %
# TODO what if they dont start with % and contain more than one % ? -> up to first % prunable, but is it used?
def is_useful(row):
    description = row['DESCRIPTION']
    if "ColumnLike" in description:
        words = description.split()
        like_criteria = words[-1]
        assert "%" in like_criteria, f"LIKE operators should have an %, but found none in {like_criteria}"
        return like_criteria[1] != '%'
    elif "ExpressionEvaluator" in description and " IN " in description:
        return False
    else:
        return True
    
scans = scans[scans.apply(is_useful, axis=1)]
EXPECTED_SCAN_COUNT = len(scans)
print(f"Of those, only {len(scans)} are useful for pruning")


# TODO do we care about reference segments, or data only?    

Statistics for TPCH contain 436 table scans
Of those, only 280 operate on more than 25000 tuples
Of those, only 242 are useful for pruning


In [5]:
(scans['RUNTIME_NS'] - scans['SINGLE_RUNTIME_NS']).max()
# TODO can the actual runtime be that much greater than the runtime on the original table?

16191218

In [6]:
# Store additional statistics
# TODO keep?

assert_correct_statistics_loaded()

def round_up_to_chunksize(row):
    if row['OUTPUT_ROWS'] % CHUNK_SIZE == 0:
        return row['OUTPUT_ROWS']
    else:
        return row['OUTPUT_ROWS'] + (CHUNK_SIZE - (row['OUTPUT_ROWS'] % CHUNK_SIZE))

scans['pruned_minimum_input_rows'] = scans.apply(round_up_to_chunksize, axis=1)

scans['selectivity'] = scans['OUTPUT_ROWS'] / scans['INPUT_ROWS']
scans['actual_selectivity'] = scans['SINGLE_OUTPUT_ROWS'] / scans['SINGLE_INPUT_ROWS']

scans['time_per_ir'] = scans['INPUT_ROWS'] / scans['RUNTIME_NS']
scans['time_per_or'] = scans['OUTPUT_ROWS'] / scans['RUNTIME_NS']

# optimal runtime assuming perfect pruning, but not sortedness
scans['optimal_runtime'] = scans['time_per_ir'] * scans['pruned_minimum_input_rows']
scans['runtime_gain'] = scans['RUNTIME_NS'] - scans['optimal_runtime']


# log runtime for sorted columns
scans['log_runtime'] = np.log2(scans['RUNTIME_NS'])
scans['optimal_log_runtime'] = np.log2(1+scans['optimal_runtime'])
scans

,QUERY_HASH,COLUMN_TYPE,TABLE_NAME,COLUMN_NAME,INPUT_ROWS,OUTPUT_ROWS,RUNTIME_NS,DESCRIPTION,SINGLE_INPUT_ROWS,SINGLE_OUTPUT_ROWS,SINGLE_RUNTIME_NS,pruned_minimum_input_rows,selectivity,actual_selectivity,time_per_ir,time_per_or,optimal_runtime,runtime_gain,log_runtime,optimal_log_runtime
7,6730c267d3eac48a,DATA,orders,o_orderstatus,1500000,729413,8364864,TableScan Impl: ColumnVsValue o_orderstatus = 'F',1500000,729413,10172682,750000,0.486275,0.486275,0.179322,0.087200,134491.128607,8.230373e+06,22.995911,17.037162
9,6ec3126b032024be,DATA,orders,o_orderstatus,1500000,729413,8210994,TableScan Impl: ColumnVsValue o_orderstatus = 'F',1500000,729413,9171234,750000,0.486275,0.486275,0.182682,0.088834,137011.426388,8.073983e+06,22.969125,17.063947
11,7324393c05ab5301,DATA,orders,o_orderstatus,1500000,729413,8179494,TableScan Impl: ColumnVsValue o_orderstatus = 'F',1500000,729413,9115287,750000,0.486275,0.486275,0.183385,0.089176,137539.070265,8.041955e+06,22.963580,17.069492
13,37e2ba0a1c4e865f,DATA,orders,o_orderstatus,1500000,729413,8236348,TableScan Impl: ColumnVsValue o_orderstatus = 'F',1500000,729413,9098455,750000,0.486275,0.486275,0.182120,0.088560,136589.663283,8.099758e+06,22.973573,17.059499
15,a17cb368eadced8f,DATA,orders,o_orderstatus,1500000,729413,8533123,TableScan Impl: ColumnVsValue o_orderstatus = 'F',1500000,729413,9112772,750000,0.486275,0.486275,0.175786,0.085480,131839.187130,8.401284e+06,23.024642,17.008431
17,bfb403aee0d212a,DATA,orders,o_orderstatus,1500000,729413,8391895,TableScan Impl: ColumnVsValue o_orderstatus = 'F',1500000,729413,9085813,750000,0.486275,0.486275,0.178744,0.086919,134057.921363,8.257837e+06,23.000565,17.032508
19,98aa70b345defa5b,DATA,part,p_name,200000,2233,380729,TableScan Impl: ColumnBetween p_name BETWEEN U...,200000,2233,427037,25000,0.011165,0.011165,0.525308,0.005865,13132.700687,3.675963e+05,18.538405,13.680986
20,445e33c234f835dc,DATA,part,p_name,200000,2190,453787,TableScan Impl: ColumnBetween p_name BETWEEN U...,200000,2190,329951,25000,0.010950,0.010950,0.440735,0.004826,11018.385278,4.427686e+05,18.791656,13.427756
21,ec4f5aa44b3ce5b2,DATA,orders,o_orderdate,1500000,57311,3291410,TableScan Impl: ColumnBetween o_orderdate BETW...,1500000,57311,3703361,75000,0.038207,0.038207,0.455732,0.017412,34179.880355,3.257230e+06,21.650274,15.060902
23,b6e90b19f6849df0,DATA,part,p_type,200000,1312,237685,TableScan Impl: ColumnVsValue p_type = 'STANDA...,200000,1312,262271,25000,0.006560,0.006560,0.841450,0.005520,21036.245451,2.166488e+05,17.858691,14.360658


In [7]:
class AbstractModel:
    
    def __init__(self, table_scans, correlations={}):
        self.table_scans = table_scans
        self.correlations = correlations
        
    def extract_interesting_columns(self):
        return list(self.table_scans['COLUMN_NAME'].unique())

    # return a list of possible clusterings
    def suggest_clustering(self, first_k=1):
        raise NotImplemented()

In [8]:
class SimpleModel(AbstractModel):
    
    def __init__(self, table_scans, correlations = {}):
        super().__init__(table_scans, correlations)        
    
    def suggest_clustering(self, first_k=1):
        interesting_columns = self.extract_interesting_columns()

        pairs = itertools.product(interesting_columns, interesting_columns)                
        total_runtimes = [self.estimate_total_runtime(self.table_scans, clustering_columns) for clustering_columns in pairs]
        total_runtimes.sort(key=lambda x: x[1], reverse=False)
        
        return total_runtimes[0:first_k]
        
    
    def estimate_total_runtime(self, single_table, clustering_columns):
        total_runtime = 0
        
        pruning_col = clustering_columns[0]
        sorted_col = clustering_columns[1]
        def compute_runtime(row):
            col_name = row['COLUMN_NAME']
            if pruning_col == sorted_col:
                if col_name == pruning_col:
                    return row['optimal_log_runtime']
                else:
                    if col_name in self.correlations.get(pruning_col, []):
                        # correlated to pruning column -> a lot of pruning, no sortedness
                        # TODO: better measure correlation
                        return 1.2 * row['optimal_runtime']
                    else:
                        return row['RUNTIME_NS']

            else:
                if col_name == pruning_col:
                    return row['optimal_runtime']
                elif col_name == sorted_col:
                    # TODO: should this be affected by correlation?
                    # we will get less chunks, so a linear scan should be close to optimal_runtime,
                    # but log time should beat it anyway
                    return row['log_runtime']
                else:
                    if col_name in self.correlations.get(pruning_col, []):
                        # correlated to pruning column -> a lot of pruning, no sortedness
                        # TODO: better measure correlation
                        return 1.2 * row['optimal_runtime']
                    else:
                        return row['RUNTIME_NS']
                    
        effective_runtime = single_table.apply(compute_runtime, axis=1)
        return [clustering_columns, effective_runtime.sum()]

In [9]:
class SingleTableMdcModel(AbstractModel):
    
    def __init__(self, table_scans, table_size, correlations = {}):
        super().__init__(table_scans, correlations)
        self.table_size = table_size
    
    def suggest_clustering(self, first_k=1):
        interesting_columns = self.extract_interesting_columns()

        pairs = itertools.product(interesting_columns, interesting_columns)
        pairs = filter(lambda x: x[0] <= x[1], pairs)
        total_runtimes = [self.estimate_total_runtime(clustering_columns) for clustering_columns in pairs]
        total_runtimes.sort(key=lambda x: x[1], reverse=False)
        
        return total_runtimes[0:first_k]
        
    
    def estimate_total_runtime(self, clustering_columns):
        def compute_unprunable_parts(row):
            if row['COLUMN_NAME'] in clustering_columns:
                return row['actual_selectivity']
            else:
                return 1.0
        
        total_runtime = 0

        # TODO kill/deal with OR
        scans_per_query = self.table_scans.sort_values(['INPUT_ROWS'], ascending=False).groupby(['QUERY_HASH'])
        for _query_hash, scans in scans_per_query:
            # TODO: kinda unrealistic assumption: everything not in the table scan result can be pruned                                    
            unprunable_parts = scans.apply(compute_unprunable_parts, axis=1)
            unprunable_part = unprunable_parts.product()
            estimated_pruned_table_size = unprunable_part * self.table_size
            
            first_row = scans.iloc[0]
            query_runtime = scans.RUNTIME_NS.sum()
            query_runtime -= first_row['RUNTIME_NS']
            query_runtime += first_row['time_per_ir'] * estimated_pruned_table_size
            # TODO modify input sizes of subsequent 
            
            total_runtime += query_runtime
            
        return [clustering_columns, total_runtime]

In [10]:
assert_correct_statistics_loaded()

def extract_single_table(table_scans, table_name):
    return table_scans[table_scans['TABLE_NAME'] == table_name]

def get_table_names(table_scans):
    return table_scans['TABLE_NAME'].unique()

clustering = {}
correlations = {
    'l_shipdate': ['l_receiptdate', 'l_commitdate'],
    'l_receiptdate': ['l_shipdate', 'l_commitdate'],
    'l_commitdate': ['l_receiptdate', 'l_shipdate']
}


table_names = get_table_names(scans)
for table_name in table_names:
    single_table = extract_single_table(scans, table_name)
    model = SingleTableMdcModel(single_table, table_sizes[table_name], correlations)
    clustering[table_name] = model.suggest_clustering(3)

clustering

{'orders': [[('o_orderdate', 'o_orderstatus'), 3011766.41869207],
  [('o_orderdate', 'o_orderdate'), 3845570.7292072806],
  [('o_orderstatus', 'o_orderstatus'), 20263401.90201918]],
 'part': [[('p_size', 'p_type'), 41150707.38212104],
  [('p_container', 'p_type'), 41535616.44631842],
  [('p_name', 'p_type'), 41742356.34591395]],
 'lineitem': [[('l_shipdate', 'l_shipmode'), 218039705.9422745],
  [('l_receiptdate', 'l_shipdate'), 220872779.594693],
  [('l_discount', 'l_shipdate'), 236601457.49311274]],
 'customer': [[('c_mktsegment', 'c_mktsegment'), 72804.84343363583]]}

In [11]:
groups = scans.groupby(['TABLE_NAME', 'QUERY_HASH'])
for group, df in groups:
    if len(df) > 1:
        print(df.columns)


Index(['QUERY_HASH', 'COLUMN_TYPE', 'TABLE_NAME', 'COLUMN_NAME', 'INPUT_ROWS',
       'OUTPUT_ROWS', 'RUNTIME_NS', 'DESCRIPTION', 'SINGLE_INPUT_ROWS',
       'SINGLE_OUTPUT_ROWS', 'SINGLE_RUNTIME_NS', 'pruned_minimum_input_rows',
       'selectivity', 'actual_selectivity', 'time_per_ir', 'time_per_or',
       'optimal_runtime', 'runtime_gain', 'log_runtime',
       'optimal_log_runtime'],
      dtype='object')
Index(['QUERY_HASH', 'COLUMN_TYPE', 'TABLE_NAME', 'COLUMN_NAME', 'INPUT_ROWS',
       'OUTPUT_ROWS', 'RUNTIME_NS', 'DESCRIPTION', 'SINGLE_INPUT_ROWS',
       'SINGLE_OUTPUT_ROWS', 'SINGLE_RUNTIME_NS', 'pruned_minimum_input_rows',
       'selectivity', 'actual_selectivity', 'time_per_ir', 'time_per_or',
       'optimal_runtime', 'runtime_gain', 'log_runtime',
       'optimal_log_runtime'],
      dtype='object')
Index(['QUERY_HASH', 'COLUMN_TYPE', 'TABLE_NAME', 'COLUMN_NAME', 'INPUT_ROWS',
       'OUTPUT_ROWS', 'RUNTIME_NS', 'DESCRIPTION', 'SINGLE_INPUT_ROWS',
       'SINGLE_OUTPUT

Outdated code fragments (older model versions) are kept below.

In [12]:
GAIN_COLUMN = 'runtime_gain'

scans_groupby_columnname = scans.groupby(['TABLE_NAME', 'COLUMN_NAME'])
sum_of_gains = pd.DataFrame(scans_groupby_columnname[GAIN_COLUMN].sum())
sum_of_gains.sort_values(by=['TABLE_NAME', GAIN_COLUMN], ascending=[True, False])

runtime_gain
TABLE_NAME COLUMN_NAME                
customer   c_mktsegment   4.892255e+06
lineitem   l_shipdate     7.949701e+08
           l_receiptdate  1.640305e+08
           l_shipmode     1.011403e+08
           l_discount     6.533017e+07
           l_quantity     3.576752e+07
orders     o_orderdate    1.781228e+08
           o_orderstatus  4.910519e+07
part       p_type         2.956403e+07
           p_brand        2.344154e+07
           p_name         3.980304e+06
           p_container    3.135618e+06
           p_size         2.304547e+06

In [13]:
assert_correct_statistics_loaded()

if BENCHMARK == "TPCH":
    TABLE = "lineitem"
else:    
    TABLE = "customer_demographics"

import itertools

def extract_single_table(table_name):
    return scans[scans['TABLE_NAME'] == table_name]

def extract_interesting_columns(df):
    return list(df['COLUMN_NAME'].unique())


correlations = {
    'l_shipdate': ['l_receiptdate', 'l_commitdate'],
    'l_receiptdate': ['l_shipdate', 'l_commitdate'],
    'l_commitdate': ['l_receiptdate', 'l_shipdate']
}
#correlations = {}
def table_sorting_options(table_name):
    single_table = extract_single_table(table_name)
    interesting_cols = extract_interesting_columns(single_table)
    pairs = itertools.product(interesting_cols, interesting_cols)
    
    total_times = []
    for pair in pairs:
        pruning_col = pair[0]
        sorted_col = pair[1]

        def compute_runtime(row):
            col_name = row['COLUMN_NAME']
            if pruning_col == sorted_col:
                if col_name == pruning_col:
                    return row['optimal_log_runtime']
                else:
                    if col_name in correlations.get(pruning_col, []):
                        # correlated to pruning column -> a lot of pruning, no sortedness
                        # TODO: better measure correlation
                        return 1.2 * row['optimal_runtime']
                    else:
                        return row['RUNTIME_NS']

            else:
                if col_name == pruning_col:
                    return row['optimal_runtime']
                elif col_name == sorted_col:
                    # TODO: should this be affected by correlation?
                    # we will get less chunks, so a linear scan should be close to optimal_runtime,
                    # but log time should beat it anyway
                    return row['log_runtime']
                else:
                    if col_name in correlations.get(pruning_col, []):
                        # correlated to pruning column -> a lot of pruning, no sortedness
                        # TODO: better measure correlation
                        return 1.2 * row['optimal_runtime']
                    else:
                        return row['RUNTIME_NS']

        effective_runtime = single_table.apply(compute_runtime, axis=1)
        total_times.append([pair, effective_runtime.sum()])    
    total_times = pd.DataFrame(total_times, columns=['columns', 'time'])    
    return total_times

options = table_sorting_options(TABLE)
options.sort_values(by=['time'], ascending=True)

,columns,time
3,"(l_shipdate, l_shipmode)",1.201590e+08
23,"(l_receiptdate, l_shipmode)",1.224453e+08
2,"(l_shipdate, l_discount)",1.562392e+08
22,"(l_receiptdate, l_discount)",1.585255e+08
1,"(l_shipdate, l_quantity)",1.860171e+08
21,"(l_receiptdate, l_quantity)",1.883034e+08
20,"(l_receiptdate, l_shipdate)",2.064296e+08
0,"(l_shipdate, l_shipdate)",2.070929e+08
4,"(l_shipdate, l_receiptdate)",2.178605e+08
24,"(l_receiptdate, l_receiptdate)",2.208103e+08


In [14]:
aggregates = pd.read_csv(f"{STATISTICS_PATH}/aggregates.csv", sep=',')

# it looks like column names are mixed up.
# COLUMN_NAME -> actually GROUP_BY_COLUMN_COUNT
# GROUP_BY_COLUMN_COUNT -> actually AGGREGATE_COLUMN_COUNT
# AGGREGATE_COLUMN_COUNT -> actually COLUMN_NAME

COL_NAME = 'AGGREGATE_COLUMN_COUNT'
GROUPBY_COL = 'COLUMN_NAME'
AGG_COL = 'GROUP_BY_COLUMN_COUNT'

# All aggregates have to read the entire table, so we cannot skip chunks.
# But getting all groups consecutive could provide a speedup
# As a result, we care only about aggregates with group by columns

interesting_aggregates = aggregates[aggregates[GROUPBY_COL] > 0]
stats = interesting_aggregates.groupby(['TABLE_NAME', COL_NAME])
out_columns = pd.DataFrame(stats['OUTPUT_ROWS'].max())
out_columns.sort_values(by=['TABLE_NAME', 'OUTPUT_ROWS'], ascending=[True, False])
aggregates[aggregates['COLUMN_TYPE'] == 'DATA']

,QUERY_HASH,AGGREGATE_HASH,COLUMN_TYPE,TABLE_NAME,COLUMN_NAME,GROUP_BY_COLUMN_COUNT,AGGREGATE_COLUMN_COUNT,INPUT_ROWS,OUTPUT_ROWS,RUNTIME_NS,DESCRIPTION
